# 02 — Data Cleaning and Preparation

This notebook focuses on cleaning and preparing the Toronto Traffic Collisions dataset.
The steps include handling missing values, fixing data types, standardizing variables,
removing duplicates, and saving a cleaned dataset for further analysis.


In [3]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)


## 1. Load Raw Dataset

The raw dataset is loaded from the `data/raw` directory.


In [5]:
df_raw = pd.read_csv("../data/raw/Traffic_Collisions_Data.csv")
print("Raw shape:", df_raw.shape)
df_raw.head()


Raw shape: (772516, 23)


,OBJECTID,EVENT_UNIQUE_ID,OCC_DATE,OCC_MONTH,OCC_DOW,OCC_YEAR,OCC_HOUR,DIVISION,FATALITIES,INJURY_COLLISIONS,FTR_COLLISIONS,PD_COLLISIONS,HOOD_158,NEIGHBOURHOOD_158,LONG_WGS84,LAT_WGS84,AUTOMOBILE,MOTORCYCLE,PASSENGER,BICYCLE,PEDESTRIAN,x,y
0,1,GO-20148000028,1/1/2014 5:00:00 AM,January,Wednesday,2014,17,D53,0,NO,YES,NO,099,Mount Pleasant East (99),-79.377616,43.701225,YES,NO,NO,NO,NO,-8.836276e+06,5.419322e+06
1,2,GO-20148004875,1/1/2014 5:00:00 AM,January,Wednesday,2014,14,D32,0,NO,NO,YES,105,Lawrence Park North (105),-79.397589,43.726091,YES,NO,NO,NO,NO,-8.838499e+06,5.423152e+06
2,3,GO-20141260499,1/1/2014 5:00:00 AM,January,Wednesday,2014,2,NSA,0,YES,NO,NO,NSA,NSA,0.000000,0.000000,YES,NO,NO,NO,NO,6.327780e-09,5.664924e-09
3,4,GO-20141260663,1/1/2014 5:00:00 AM,January,Wednesday,2014,3,NSA,0,NO,NO,YES,NSA,NSA,0.000000,0.000000,YES,NO,NO,NO,NO,6.327780e-09,5.664924e-09
4,5,GO-20141261162,1/1/2014 5:00:00 AM,January,Wednesday,2014,5,NSA,0,YES,NO,NO,NSA,NSA,0.000000,0.000000,YES,NO,NO,NO,NO,6.327780e-09,5.664924e-09


## 2. Pre-cleaning Validation
We check duplicates, data types, and missing values before applying any changes.


In [17]:
duplicates_before = df_raw.duplicated().sum()
duplicates_before



0

In [7]:
df_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 772516 entries, 0 to 772515
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   OBJECTID           772516 non-null  int64  
 1   EVENT_UNIQUE_ID    772516 non-null  object 
 2   OCC_DATE           772516 non-null  object 
 3   OCC_MONTH          772516 non-null  object 
 4   OCC_DOW            772516 non-null  object 
 5   OCC_YEAR           772516 non-null  int64  
 6   OCC_HOUR           772516 non-null  int64  
 7   DIVISION           772516 non-null  object 
 8   FATALITIES         772516 non-null  int64  
 9   INJURY_COLLISIONS  772512 non-null  object 
 10  FTR_COLLISIONS     772512 non-null  object 
 11  PD_COLLISIONS      772512 non-null  object 
 12  HOOD_158           772516 non-null  object 
 13  NEIGHBOURHOOD_158  772516 non-null  object 
 14  LONG_WGS84         772516 non-null  float64
 15  LAT_WGS84          772516 non-null  float64
 16  AU

In [18]:
missing_before = df_raw.isna().sum().sort_values(ascending=False)
missing_before.head(15)


PD_COLLISIONS        4
PEDESTRIAN           4
BICYCLE              4
PASSENGER            4
MOTORCYCLE           4
AUTOMOBILE           4
INJURY_COLLISIONS    4
FTR_COLLISIONS       4
NEIGHBOURHOOD_158    0
x                    0
LAT_WGS84            0
LONG_WGS84           0
OBJECTID             0
HOOD_158             0
EVENT_UNIQUE_ID      0
dtype: int64

## 3. Cleaning Process

A working copy of the dataset is created. All cleaning steps are applied only to this copy.


In [ ]:
df = df_raw.copy()


### 3.1 Convert Date Column

The `OCC_DATE` column is converted to datetime format to support time-based analysis.
Invalid dates are coerced to missing values.


In [11]:
df["OCC_DATE"] = pd.to_datetime(df["OCC_DATE"], errors="coerce")
df["OCC_DATE"].isna().sum()


0

### 3.2 Standardize Binary (YES/NO) Columns

Binary indicator columns are standardized by trimming whitespace,
converting to uppercase, and handling missing values.


In [19]:
yes_no_cols = [
    "INJURY_COLLISIONS", "FTR_COLLISIONS", "PD_COLLISIONS",
    "AUTOMOBILE", "MOTORCYCLE", "PASSENGER",
    "BICYCLE", "PEDESTRIAN", "TRUCK"
]

yes_no_cols = [c for c in yes_no_cols if c in df.columns]

for c in yes_no_cols:
    df[c] = df[c].astype("string").str.strip().str.upper()
    df[c] = df[c].fillna("UNKNOWN")

df[yes_no_cols].head()


,INJURY_COLLISIONS,FTR_COLLISIONS,PD_COLLISIONS,AUTOMOBILE,MOTORCYCLE,PASSENGER,BICYCLE,PEDESTRIAN
0,NO,YES,NO,YES,NO,NO,NO,NO
1,NO,NO,YES,YES,NO,NO,NO,NO
2,YES,NO,NO,YES,NO,NO,NO,NO
3,NO,NO,YES,YES,NO,NO,NO,NO
4,YES,NO,NO,YES,NO,NO,NO,NO


### 3.3 Handle Missing Values in Categorical Columns

Missing values in categorical columns are replaced with "UNKNOWN"
to preserve records without introducing false information.


In [20]:
cat_cols = [
    "DIVISION", "NEIGHBOURHOOD_158", "HOOD_158",
    "OCC_MONTH", "OCC_DOW"
]

cat_cols = [c for c in cat_cols if c in df.columns]

for c in cat_cols:
    df[c] = df[c].astype("string").str.strip()
    df[c] = df[c].fillna("UNKNOWN")

df[cat_cols].head()



,DIVISION,NEIGHBOURHOOD_158,HOOD_158,OCC_MONTH,OCC_DOW
0,D53,Mount Pleasant East (99),099,January,Wednesday
1,D32,Lawrence Park North (105),105,January,Wednesday
2,NSA,NSA,NSA,January,Wednesday
3,NSA,NSA,NSA,January,Wednesday
4,NSA,NSA,NSA,January,Wednesday


### 3.4 Duplicate Removal

Exact duplicate rows are removed to improve data quality.



In [21]:
rows_before = df.shape[0]
df = df.drop_duplicates()
rows_after = df.shape[0]

print("Rows before:", rows_before)
print("Rows after:", rows_after)
print("Duplicates removed:", rows_before - rows_after)


Rows before: 772516
Rows after: 772516
Duplicates removed: 0


## 4. Post-Cleaning Validation

This step confirms the effectiveness of the cleaning process
by re-checking data types, missing values, and dataset size.


In [22]:
print("Clean dataset shape:", df.shape)
df.info()



Clean dataset shape: (772516, 23)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 772516 entries, 0 to 772515
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   OBJECTID           772516 non-null  int64         
 1   EVENT_UNIQUE_ID    772516 non-null  object        
 2   OCC_DATE           772516 non-null  datetime64[ns]
 3   OCC_MONTH          772516 non-null  string        
 4   OCC_DOW            772516 non-null  string        
 5   OCC_YEAR           772516 non-null  int64         
 6   OCC_HOUR           772516 non-null  int64         
 7   DIVISION           772516 non-null  string        
 8   FATALITIES         772516 non-null  int64         
 9   INJURY_COLLISIONS  772516 non-null  string        
 10  FTR_COLLISIONS     772516 non-null  string        
 11  PD_COLLISIONS      772516 non-null  string        
 12  HOOD_158           772516 non-null  string        
 13  NEIGHBOURH

In [23]:
missing_after = df.isna().sum().sort_values(ascending=False)
missing_after.head(15)


OBJECTID             0
HOOD_158             0
x                    0
PEDESTRIAN           0
BICYCLE              0
PASSENGER            0
MOTORCYCLE           0
AUTOMOBILE           0
LAT_WGS84            0
LONG_WGS84           0
NEIGHBOURHOOD_158    0
PD_COLLISIONS        0
EVENT_UNIQUE_ID      0
FTR_COLLISIONS       0
INJURY_COLLISIONS    0
dtype: int64

## 5. Save Cleaned Dataset

The cleaned dataset is saved to the `data/cleaned` directory.
Due to file size, this file is not pushed to GitHub.


In [24]:
clean_path = "../data/cleaned/Traffic_Collisions_Cleaned.csv"
df.to_csv(clean_path, index=False)

clean_path


'../data/cleaned/Traffic_Collisions_Cleaned.csv'

## 6. Cleaning Summary

- Converted OCC_DATE to datetime format
- Standardized binary (YES/NO) columns
- Handled missing categorical values using "UNKNOWN"
- Removed duplicate records
- Saved a cleaned dataset for exploratory analysis and modeling
